# Boston House Prices Prediction

In [1]:
import numpy as np
import pandas as pd
import missingno
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import xgboost as xg
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

In [2]:
### Fetching the dataset

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataset = pd.read_csv('../input/boston-house-prices/housing.csv', header = None, delimiter = r"\s+", names = column_names)

In [3]:
dataset.head(10)

In [4]:
dataset.shape

In [5]:
dataset.info()

Here, the columns - CHAS and RAD are categorical. Hence, we modify the datatype of these columns to category.

In [6]:
dataset.CHAS = dataset.CHAS.astype('category')
dataset.RAD = dataset.RAD.astype('category')

Looking at the modified datatypes of the columns in the dataset.

In [7]:
dataset.info()

From the above data, it is evident that there are no missing values in the dataset. Let's verify that.

In [8]:
### Missing data by columns in the dataset

dataset.isnull().sum().sort_values(ascending = False)

Here, we can see that there are no missing values in the dataset.

In [9]:
### Summary statistics of the numerical columns in the dataset

dataset.describe()

In [10]:
### Value counts of the column - CHAS

chas_count = dataset['CHAS'].value_counts(dropna = False)
chas_count

In [11]:
### Bar graph showing the value counts of the column - CHAS

sns.barplot(chas_count.index, chas_count.values, alpha = 0.8)
plt.title('Bar graph showing the value counts of the column - CHAS')
plt.ylabel('Number of Occurrences', fontsize = 12)
plt.xlabel('CHAS', fontsize = 12)
plt.show()

From the above graph, we can see that most of the values of the column CHAS is 0.

In [12]:
### Mean price per each CHAS 

mean_price_chas = dataset[['CHAS', 'MEDV']].groupby('CHAS', as_index = False).mean()
mean_price_chas

In [13]:
### Mean Sales Price for each CHAS

sns.barplot(mean_price_chas['CHAS'], mean_price_chas['MEDV'], alpha = 0.8)
plt.title('Mean Sales Price for each CHAS')
plt.ylabel('Mean Sales Price', fontsize = 12)
plt.xlabel('CHAS', fontsize = 12)
plt.show()

From the above graph, we can see that the mean sales price of a house is similar in both the houses that are either river bound or not.

In [14]:
### Value counts of the column - RAD

rad_count = dataset['RAD'].value_counts(dropna = False)
rad_count

In [15]:
### Bar graph showing the value counts of the column - RAD

sns.barplot(rad_count.index, rad_count.values, alpha = 0.8)
plt.title('Bar graph showing the value counts of the column - RAD')
plt.ylabel('Number of Occurrences', fontsize = 12)
plt.xlabel('RAD', fontsize = 12)
plt.show()

From the above graph, we can see that there are 9 different values for the column - RAD.

In [16]:
### Mean price per each RAD 

mean_price_rad = dataset[['RAD', 'MEDV']].groupby('RAD', as_index = False).mean()
mean_price_rad

In [17]:
### Mean Sales Price for each RAD

sns.barplot(mean_price_rad['RAD'], mean_price_rad['MEDV'], alpha = 0.8)
plt.title('Mean Sales Price for each RAD')
plt.ylabel('Mean Sales Price', fontsize = 12)
plt.xlabel('RAD', fontsize = 12)
plt.show()

Here, we can see that the mean sales price based on RAD is almost similar for each RAD type except 24. We will use this observation during Feature Engineering where we focus on reducing the number of different values for RAD.

In [18]:
### Understanding the distribution of the column - MEDV

sns.distplot(dataset['MEDV'], label = 'Skewness: %.2f'%(dataset['MEDV'].skew()))
plt.legend(loc = 'best')
plt.title('Median Value Distribution')

From the above graph, we can see that the distribution is close to normal distribution with a slight skewness.

In [19]:
### Understanding the distribution of the column - CRIM

sns.distplot(dataset['CRIM'], label = 'Skewness: %.2f'%(dataset['CRIM'].skew()))
plt.legend(loc = 'best')
plt.title('Per Capita Crime Rate By Town')

From the above graph, we can see that there is a high degree of skewness in the distribution of the column - CRIM.

In [20]:
### Understanding the distribution of the column - ZN

sns.distplot(dataset['ZN'], label = 'Skewness: %.2f'%(dataset['ZN'].skew()))
plt.legend(loc = 'best')
plt.title('Proportion of Residential Land Zones')

From the above graph, we can see that there is a high degree of skewness in the distribution of the column - ZN.

In [21]:
### Understanding the distribution of the column - INDUS

sns.distplot(dataset['INDUS'], label = 'Skewness: %.2f'%(dataset['INDUS'].skew()))
plt.legend(loc = 'best')
plt.title('Proportion of Non - Retail Business Acres Per Town')

From the above graph, we can see that there are 2 modes for the column - INDUS.

In [22]:
### Understanding the distribution of the column - NOX

sns.distplot(dataset['NOX'], label = 'Skewness: %.2f'%(dataset['NOX'].skew()))
plt.legend(loc = 'best')
plt.title('Nitric Oxide Concentration - Distribution')

From the above graph, we can see that the above distribution has a less degree of skewness.

In [23]:
### Understanding the distribution of the column - RM

sns.distplot(dataset['RM'], label = 'Skewness: %.2f'%(dataset['RM'].skew()))
plt.legend(loc = 'best')
plt.title('Average number of rooms per dwelling')

From the above graph, we can see that the distribution is close to normal distribution with a very slight skewness.

In [24]:
### Understanding the distribution of the column - AGE

sns.distplot(dataset['AGE'], label = 'Skewness: %.2f'%(dataset['AGE'].skew()))
plt.legend(loc = 'best')
plt.title('Proportion of Owner Occupied Units')

From the above graph, we can see that the distribution is slightly left skewed.

In [25]:
### Understanding the distribution of the column - DIS

sns.distplot(dataset['DIS'], label = 'Skewness: %.2f'%(dataset['DIS'].skew()))
plt.legend(loc = 'best')
plt.title('Weighted Distances Distribution')

From the above graph, we can see that the distribution is slightly right skewed with a slight degree of skewness.

In [26]:
### Understanding the distribution of the column - TAX

sns.distplot(dataset['TAX'], label = 'Skewness: %.2f'%(dataset['TAX'].skew()))
plt.legend(loc = 'best')
plt.title('Full Value Property Tax Distribution')

From the above graph, we can see that there are 2 modes for the column - TAX.

In [27]:
### Understanding the distribution of the column - PTRATIO

sns.distplot(dataset['PTRATIO'], label = 'Skewness: %.2f'%(dataset['PTRATIO'].skew()))
plt.legend(loc = 'best')
plt.title('Pupil Teacher Ratio By Town')

From the above graph, we can see that the distribution is slightly left skewed.

In [28]:
### Understanding the distribution of the column - B

sns.distplot(dataset['B'], label = 'Skewness: %.2f'%(dataset['B'].skew()))
plt.legend(loc = 'best')
plt.title('Proportion of Blacks By Town')

From the above graph, we can see that the distribution is highly left skewed.

In [29]:
### Understanding the distribution of the column - LSTAT

sns.distplot(dataset['LSTAT'], label = 'Skewness: %.2f'%(dataset['LSTAT'].skew()))
plt.legend(loc = 'best')
plt.title('Distribution of the Lower Status of the Population')

From the above graph, we can see that the distribution is slightly right skewed.

In [30]:
def detect_outliers(df, n, features_list):
    outlier_indices = [] 
    for feature in features_list: 
        Q1 = np.percentile(df[feature], 25)
        Q3 = np.percentile(df[feature], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR 
        outlier_list_col = df[(df[feature] < Q1 - outlier_step) | (df[feature] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col) 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(key for key, value in outlier_indices.items() if value > n) 
    return multiple_outliers

outliers_to_drop = detect_outliers(dataset, 2, ['MEDV', 'CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 
                                               'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
print("We will drop these {} indices: ".format(len(outliers_to_drop)), outliers_to_drop)

Now let's look at the data present in the rows.

In [31]:
dataset.iloc[outliers_to_drop, :]

We will drop these rows from the dataset.

In [32]:
### Drop outliers and reset index

print("Before: {} rows".format(len(dataset)))
dataset = dataset.drop(outliers_to_drop, axis = 0).reset_index(drop = True)
print("After: {} rows".format(len(dataset)))

In [33]:
### Lets look at the new dataset

dataset

The column RiverBound consists of two variables - Yes or No based on the column - CHAS. If CHAS value is 1, then RiverBound is Yes; else No.

In [34]:
### Fetching the data - CHAS

chas_data = list(dataset['CHAS'])

### Creating new column - RiverBound

river_bound = []
for value in chas_data:
    if value == 1:
        river_bound.append('Yes')
    else:
        river_bound.append('No')

In [35]:
### Adding the river_bound data to RiverBound column of the dataset

dataset['RiverBound'] = river_bound
dataset

In [36]:
### Dropping the column - CHAS from the dataset

dataset.drop(['CHAS'], axis = 1, inplace = True)
dataset

Here, we will modify the RAD column to 2 new classes. One belonging to RAD value 24 and other belonging to remaining all RAD values.

In [37]:
### Fetching the data - RAD

rad_data = list(dataset['RAD'])

### Creating new column - NewRAD

new_rad = []
for value in rad_data:
    if value == 24:
        new_rad.append(1)
    else:
        new_rad.append(2)

In [38]:
### Adding the new_rad data to NewRAD column of the dataset

dataset['NewRAD'] = new_rad
dataset

In [39]:
### Dropping the column - RAD from the dataset

dataset.drop(['RAD'], axis = 1, inplace = True)
dataset

Since, the Tax column is bi-modal with clear seperation between the data, we will divide the data into 2 categories and check if the Sales Price of a house varies with each class.

In [40]:
### Fetching the data - TAX

tax_data = list(dataset['TAX'])

### Creating new column - TaxSlab

tax_slab = []
for value in tax_data:
    if value <= np.median(tax_data):
        tax_slab.append('Class 1')
    else:
        tax_slab.append('Class 2')

In [41]:
### Adding the tax_slab data to TaxSlab column of the dataset

dataset['TaxSlab'] = tax_slab
dataset

In [42]:
### Value counts of the column - TaxSlab

tax_slab_count = dataset['TaxSlab'].value_counts(dropna = False)
tax_slab_count

In [43]:
### Bar graph showing the value counts of the column - TaxSlab

sns.barplot(tax_slab_count.index, tax_slab_count.values, alpha = 0.8)
plt.title('Bar graph showing the value counts of the column - TaxSlab')
plt.ylabel('Number of Occurrences', fontsize = 12)
plt.xlabel('TaxSlab', fontsize = 12)
plt.show()

In [44]:
### Mean price per each RAD 

mean_price_tax_slab = dataset[['TaxSlab', 'MEDV']].groupby('TaxSlab', as_index = False).mean()
mean_price_tax_slab

In [45]:
### Mean Sales Price for each TaxSlab

sns.barplot(mean_price_tax_slab['TaxSlab'], mean_price_tax_slab['MEDV'], alpha = 0.8)
plt.title('Mean Sales Price for each TaxSlab')
plt.ylabel('Mean Sales Price', fontsize = 12)
plt.xlabel('TaxSlab', fontsize = 12)
plt.show()

From the above graph, we can see that the Sales Price is lesser for the houses that belong to Class 2.

In [46]:
### Dropping the column - TAX from the dataset

dataset.drop(['TAX'], axis = 1, inplace = True)
dataset

In [47]:
### Understanding the distribution of the column - ZN

sns.distplot(dataset['ZN'], label = 'Skewness: %.2f'%(dataset['ZN'].skew()))
plt.legend(loc = 'best')
plt.title('Proportion of Residential Land Zones')

In [48]:
### Understanding the distribution of the data Box_Cox(ZN)

zn_data = [1 if value == 0 else value for value in dataset['ZN']]

modified_zn, _ = stats.boxcox(zn_data)
dataset['ZN'] = modified_zn

sns.distplot(dataset['ZN'], label = 'Skewness: %.2f'%(dataset['ZN'].skew()))
plt.legend(loc = 'best')
plt.title('Proportion of Residential Land Zones')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

In [49]:
### Understanding the distribution of the column - CRIM

sns.distplot(dataset['CRIM'], label = 'Skewness: %.2f'%(dataset['CRIM'].skew()))
plt.legend(loc = 'best')
plt.title('Per Capita Crime Rate By Town')

In [50]:
### Understanding the distribution of the data Box_Cox(CRIM)

crim_data = [1 if value < 1 else value for value in dataset['CRIM']]

modified_crim, _ = stats.boxcox(crim_data)
dataset['CRIM'] = modified_crim

sns.distplot(dataset['CRIM'], label = 'Skewness: %.2f'%(dataset['CRIM'].skew()))
plt.legend(loc = 'best')
plt.title('Per Capita Crime Rate By Town')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

In [51]:
### A function to normalize numerical columns

def normalize_columns(dataframe, column):
    data = dataframe[column]
    mini = min(data)
    maxi = max(data)
    
    new_data = []
    for value in data:
        new_data.append((value - mini)/(maxi - mini))
    
    dataframe[column] = new_data

numerical_columns = list(dataset.columns)[: -4]
for each_column in numerical_columns:
    normalize_columns(dataset, each_column)

In [52]:
### Looking at the sample records of the dataset

dataset

Here, we will use One Hot Encoding for the columns - RiverBound, NewRAD, TaxSlab.

In [53]:
### One Hot Encoding the columns - RiverBound, NewRAD, TaxSlab of the dataset

encoded_dataset = pd.get_dummies(data = dataset, columns = ['RiverBound', 'NewRAD', 'TaxSlab'])
encoded_dataset

In [54]:
### Create the column - Target using MEDV

target_data = encoded_dataset['MEDV']
encoded_dataset['Target'] = target_data

### Dropping the column - MEDV

encoded_dataset.drop(['MEDV'], axis = 1, inplace = True)
encoded_dataset

##### Plotting the correlation matrix for the numerical columns

In [55]:
### Plotting the correlation between various columns of the filter_dataset

plt.figure(figsize = (10, 10))
heatmap = sns.heatmap(encoded_dataset.corr(), vmin = -1, vmax = 1, annot = True)
heatmap.set_title('Correlation Heatmap', fontdict = {'fontsize' : 12}, pad = 12)

From the above figure, we can see that there are a few strong correlations between the columns.

On observing the above correlations, we will drop the columns NOX, DIS, AGE from the dataset to ensure we reduce the impact of certain features over other.

Now, we will remove the columns - NOX, DIS, AGE from the encoded dataset.

In [56]:
### Dropping the columns - NOX, DIS, AGE from the encoded dataset

encoded_dataset.drop(['NOX', 'DIS', 'AGE'], axis = 1, inplace = True)
encoded_dataset

In [57]:
### Plotting the new correlation matrix

plt.figure(figsize = (10, 10))
heatmap = sns.heatmap(encoded_dataset.corr(), vmin = -1, vmax = 1, annot = True)
heatmap.set_title('Correlation Heatmap', fontdict = {'fontsize' : 12}, pad = 12)

In [58]:
### Splitting the dataset to the matrices X and Y

X = encoded_dataset.iloc[:, : -1].values
Y = encoded_dataset.iloc[:, -1].values

In [59]:
### Looking at the new training data - X

X

In [60]:
### Looking at the new test data - Y

Y

In [61]:
### Dividing the dataset into train and test in the ratio of 80 : 20

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 27, shuffle = True)

##### Applying XGBoost Regression

In [63]:
### Training the XGBoost Regression model on the Training set

xgboost_regressor = xg.XGBRegressor(objective ='reg:linear', n_estimators = 100, seed = 27)
xgboost_regressor.fit(X_train, Y_train)

In [64]:
### Predicting the Test set results

Y_pred = xgboost_regressor.predict(X_test)

In [66]:
### Calculating Mean Squared Log Error for the model

mse = round(mean_squared_error(Y_test, Y_pred), 3)
rmse = round(sqrt(mse), 3)


print('Root Mean Squared Error of the model is : {}'.format(rmse))